<a href="https://colab.research.google.com/github/ggwnsghgg/book_project/blob/main/2021%20/%2004%20/%2001%20/%20%EC%9C%A0%EC%82%AC%EB%8F%84%20%EA%B2%80%EC%A6%9D_fin.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 필요한 패키지 install

In [1]:
!pip install pymysql


# 형태소 기반 토크나이징 (Konlpy)
!python3 -m pip install konlpy
!bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)



#한국어 문장 분리모델
!pip install kss


!sudo apt-get install curl git
!bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)


#mecab 코랩서 오류 안나게!
!apt-get update
!apt-get install g++ openjdk-8-jdk 
!pip3 install konlpy JPype1-py3
!bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)

     |████████████████████████████████| 51kB 3.0MB/s 
     |████████████████████████████████| 19.4MB 2.0MB/s 
     |████████████████████████████████| 92kB 9.4MB/s 
     |████████████████████████████████| 460kB 51.6MB/s 
  Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3
Installing automake (A dependency for mecab-ko)
Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:7 https://developer.download.nvidia.com/compute/mac

In [2]:
import os
import kss
import re
import pandas as pd
import numpy as np
import konlpy
from tqdm import tqdm
from konlpy.tag import Mecab
import pymysql

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from google.colab import drive
drive.mount('/content/drive')



# #데이터 가져오기
# db = pymysql.connect(host='220.86.191.27', port=3306, user='playdata', password='playdata123!', db='bookdata', charset='utf8')

# curs = db.cursor(pymysql.cursors.DictCursor)  

# query = '''
# select *
#     from books_data'''
# df_books_data = pd.read_sql(query, db)

Mounted at /content/drive


In [3]:
removal_list =  "‘, ’, ◇, ‘, ”,  ’, ', ·, \“, ·, △, ●,  , ■, (, ), \", >>, `, /, -,∼,=,ㆍ<,>, .,?, !,【,】, …, ◆,%"

def cleansing_special(sentence):
    # 특수문자를 전처리를 하는 함수
    sentence = re.sub("[.,\'\"’‘”“!?]", "", sentence)
    sentence = re.sub("[^가-힣0-9a-zA-Z\\s]", " ", sentence)
    sentence = re.sub("\s+", " ", sentence)
    
    sentence = sentence.translate(str.maketrans(removal_list, ' '*len(removal_list)))
    sentence = sentence.strip()
    
    return sentence

def cleansing_numbers(sentence):
    # 숫자를 전처리(delexicalization) 하는 함수    
    sentence = re.sub('[0-9]+', 'NUM', sentence)
    sentence = re.sub('NUM\s+', "NUM", sentence)
    sentence = re.sub('[NUM]+', "NUM", sentence)
    
    return sentence

In [4]:
# 구글 드라이브 사용시

path = '/content/drive/MyDrive/'
# df_books = pd.read_excel(os.path.join(path, 'book_preprocessing.xlsx'))
df_books = pd.read_excel(os.path.join(path, 'book_fin.xlsx'))

In [5]:
df_books

,Unnamed: 0,index,book_id,title,genre,words,Unnamed: 6,Unnamed: 7,Unnamed: 8
0,0,0,0,요리코를 위해,추리/미스터리,리코 수 이자 위 누구 출발 탐정 사체 심판 무대 남자 진실 딸 패배 세팅 파악 괴...,NaN,NaN,1
1,1,1,1,그녀는 다 계획이 있다,추리/미스터리,미래 복고 동료 교코 호텔 원대 미스터리 직장 컴 자신 계획 터치 날 감사 파티 살...,NaN,NaN,1
2,2,2,2,블랙 쇼맨과 이름 없는 마을의 살인,추리/미스터리,소식 통화 살 순간 너 결혼식 시작 취직 통 약혼자 전화 경 생전 결혼 마지막 고향...,NaN,NaN,1
3,3,3,3,돌이킬 수 없는 약속,추리/미스터리,소설 수 야쿠 가쿠 전 작가 미스터리 죄 포상 놈 딸 제 란 마루 숨 추리 약속 주...,NaN,NaN,1
4,4,4,4,루팡의 딸,추리/미스터리,이 발생 카즈마 사체 자신 딸 코 날 대부 변 경찰 손녀 오 괴한 신분 도둑 발견 ...,NaN,NaN,1
...,...,...,...,...,...,...,...,...,...
2484,2484,2484,3617,드래건 살인사건,추리/미스터리,본성 풀 작품 알 구성 반 지적 분석 대 파일 미스터리 심리 내면 주말 특유 인우드...,NaN,NaN,1
2485,2485,2485,3618,가든 살인사건,추리/미스터리,상류층 본성 실내 내기 작품 경마 구성 반 가든 지적 분석 대 파일 미스터리 심리 ...,NaN,NaN,1
2486,2486,2486,3619,카지노 살인사건,추리/미스터리,불리 피해자 상류층 어디 맨해튼 미심 발생 꼬리 케이드 보이 미국 물 파일 몬테카를...,NaN,NaN,1
2487,2487,2487,3620,단테클럽 1,추리/미스터리,다빈치 활동 과정 작품 소설 번역 저자 미국 공로 실화 협회 단테 제작 붐 신곡 살...,NaN,NaN,1


In [ ]:
df_books.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 73508 entries, 0 to 73507
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  73508 non-null  int64  
 1   title       73508 non-null  object 
 2   author      64865 non-null  object 
 3   pub         73508 non-null  object 
 4   price       73508 non-null  int64  
 5   review      73508 non-null  int64  
 6   rating      73508 non-null  float64
 7   story       67630 non-null  object 
 8   img         65530 non-null  object 
 9   genre       73508 non-null  object 
dtypes: float64(1), int64(3), object(6)
memory usage: 5.6+ MB


In [ ]:
df_books=df_books.rename(columns = {'Unnamed: 0' : 'book_id'})
#결측값 제거
df_books = df_books.dropna()

In [ ]:
df_books.value_counts(df_books['rating'].values, sort=False)

0.0     44010
2.0        57
3.0        13
3.5         1
3.7         1
        ...  
9.6       174
9.7        78
9.8        87
9.9        15
10.0     1273
Length: 61, dtype: int64

In [ ]:
#레이팅, 리뷰값 모두 0점인것 제거 (다행히 리뷰와 레이팅이 모두 0점인게 공통적이다)
df_books =df_books[df_books.review != 0]

In [ ]:
df_books =df_books.reset_index(drop = False, inplace = False)

In [ ]:
df_books

,index,book_id,title,author,pub,price,review,rating,story,img,genre
0,0,0,요리코를 위해,노리즈키 린타로,모모,13500,43,9.2,스스로 악마 이자 신 딸 살해 남자 심판 아버지 비극 출발 이자 마지막 끝내 실체 ...,http://image.yes24.com/goods/89392792/L,추리/미스터리
1,1,1,그녀는 다 계획이 있다,히가시노 게이고,하빌리스,14220,13,9.4,하나 보석 점 살인 사건 과거 미래 사건 경쾌하다 리듬 감 있다 터치 게이고 복고 ...,http://image.yes24.com/goods/97561319/L,추리/미스터리
2,2,2,블랙 쇼맨과 이름 없는 마을의 살인,히가시노 게이고,알에이치코리아(RHK),16200,54,9.1,아버지 살해 당하다 그것 결혼 소식 직후 행복하다 순간 비극 시작 대기업 취직 후 ...,http://image.yes24.com/goods/95712763/L,추리/미스터리
3,3,3,돌이킬 수 없는 약속,야쿠마루 가쿠,북플라자,13500,217,8.2,제 에드가 란 포상 수상 작가 야쿠 마루 가쿠 미스터리 추리 소설 내 딸 살해 놈 ...,http://image.yes24.com/goods/35552641/L,추리/미스터리
4,4,4,루팡의 딸,요코제키 다이,북플라자,13500,31,8.5,도둑 집안 대부 변 사체 발견 도둑 집안 대부 이 오 하나 코 손녀 딸 있다 하나 ...,http://image.yes24.com/goods/78860364/L,추리/미스터리
...,...,...,...,...,...,...,...,...,...,...,...
11151,65517,65517,범상치 않은 관계 1,정해길,다향,8100,1,8.0,여자 키 중성 외모 만년 휴학생 송혜민 아버지 빚 때문 만난 사채 업자 로부터 가지...,http://image.yes24.com/goods/11367513/L,로맨스
11152,65524,65524,보통의 세계,권도란,스칼렛,8820,1,6.0,사고 동시 살의 기억 남자 승 왜 그 일 년 기억 왜 열렬하다 사랑 애인 유라 것 ...,http://image.yes24.com/goods/11318506/L,로맨스
11153,65525,65525,미주부동산 2,이휴정,신영미디어,8100,1,4.0,그 아프다 몸 끌 때 생각 언제 이 사람 내 눈물 때 다른 것 제 이렇다 식 저 자...,http://image.yes24.com/goods/11245157/L,로맨스
11154,65526,65526,미주부동산 1,이휴정,신영미디어,8100,1,4.0,김 콜라 얼음 아이스커피 소리 작다 노래방 마이크 저녁 간판 자동차 달리 도시 셰익...,http://image.yes24.com/goods/11245152/L,로맨스


In [ ]:
df_books.tail()

,index,book_id,title,author,pub,price,review,rating,story,img,genre
11151,65517,65517,범상치 않은 관계 1,정해길,다향,8100,1,8.0,여자 키 중성 외모 만년 휴학생 송혜민 아버지 빚 때문 만난 사채 업자 로부터 가지...,http://image.yes24.com/goods/11367513/L,로맨스
11152,65524,65524,보통의 세계,권도란,스칼렛,8820,1,6.0,사고 동시 살의 기억 남자 승 왜 그 일 년 기억 왜 열렬하다 사랑 애인 유라 것 ...,http://image.yes24.com/goods/11318506/L,로맨스
11153,65525,65525,미주부동산 2,이휴정,신영미디어,8100,1,4.0,그 아프다 몸 끌 때 생각 언제 이 사람 내 눈물 때 다른 것 제 이렇다 식 저 자...,http://image.yes24.com/goods/11245157/L,로맨스
11154,65526,65526,미주부동산 1,이휴정,신영미디어,8100,1,4.0,김 콜라 얼음 아이스커피 소리 작다 노래방 마이크 저녁 간판 자동차 달리 도시 셰익...,http://image.yes24.com/goods/11245152/L,로맨스
11155,65528,65528,나쁜 결혼,김진영,스칼렛,8100,1,4.0,저 결혼 아내 로서 의무 사랑 마음 여자 지우 그렇다 남자 아내 당신 말 의무 속 ...,http://image.yes24.com/sysimage/pd_19_L.gif,로맨스


In [ ]:
df_mystery=df_books[df_books['genre']=='추리/미스터리']
df_horror =df_books[df_books['genre']== '공포/스릴러']
df_fantasy = df_books[df_books['genre']=='판타지']
df_martial = df_books[df_books['genre']=='무협']
df_sf = df_books[df_books['genre']=='SF']
df_history = df_books[df_books['genre']=='역사']
df_romance = df_books[df_books['genre']=='로맨스']


print(df_mystery.shape)
print(df_horror.shape)
print(df_fantasy.shape)
print(df_martial.shape)
print(df_sf.shape)
print(df_history.shape)
print(df_romance.shape)

(2489, 11)
(1230, 11)
(2399, 11)
(470, 11)
(656, 11)
(1324, 11)
(2588, 11)


In [ ]:
# 형태소 작업 형태소 작업

from konlpy.tag import Mecab

df_mystery_list = list(np.array(df_mystery['story'].tolist()))
df_horror_list = list(np.array(df_horror['story'].tolist()))
df_fantasy_list = list(np.array(df_fantasy['story'].tolist()))
df_martial_list = list(np.array(df_martial['story'].tolist()))
df_sf_list = list(np.array(df_sf['story'].tolist()))
df_history_list = list(np.array(df_history['story'].tolist()))
df_romance_list = list(np.array(df_romance['story'].tolist()))

In [ ]:

mecab = Mecab()

In [ ]:

#미스터리
all_sentences_mystery = []
for idx, line in enumerate(df_mystery_list):
  #print(f"--미스테리소설{idx}번--")
  preprocessed = cleansing_numbers(line)
  preprocessed = cleansing_special(preprocessed)
  #명사
  preprocessed_list = ' '.join(list(set(mecab.nouns(preprocessed))))
  #print(preprocessed_list)
  all_sentences_mystery.append(preprocessed_list)

In [ ]:
#호러
all_sentences_horror = []
for idx, line in enumerate(df_horror_list):
  #print(f"--호러소설{idx}번--")
  preprocessed = cleansing_numbers(line)
  preprocessed = cleansing_special(preprocessed)
  #명사
  preprocessed_list = ' '.join(list(set(mecab.nouns(preprocessed))))
  #print(preprocessed_list)
  all_sentences_horror.append(preprocessed_list)

In [ ]:
#판타지
all_sentences_fantasy = []
for idx, line in enumerate(df_fantasy_list):
  #print(f"--판타지소설{idx}번--")
  preprocessed = cleansing_numbers(line)
  preprocessed = cleansing_special(preprocessed)
  #명사
  preprocessed_list = ' '.join(list(set(mecab.nouns(preprocessed))))
  #print(preprocessed_list)
  all_sentences_fantasy.append(preprocessed_list)

In [ ]:
#무협
all_sentences_martial = []
for idx, line in enumerate(df_martial_list):
  #print(f"--무협소설{idx}번--")
  preprocessed = cleansing_numbers(line)
  preprocessed = cleansing_special(preprocessed)
  #명사
  preprocessed_list = ' '.join(list(set(mecab.nouns(preprocessed))))
  #print(preprocessed_list)
  all_sentences_martial.append(preprocessed_list)

In [ ]:

#sf
all_sentences_sf = []
for idx, line in enumerate(df_sf_list):
  #print(f"--sf소설{idx}번--")
  preprocessed = cleansing_numbers(line)
  preprocessed = cleansing_special(preprocessed)
  #명사
  preprocessed_list = ' '.join(list(set(mecab.nouns(preprocessed))))
  #print(preprocessed_list)
  all_sentences_sf.append(preprocessed_list)

In [ ]:
#역사
all_sentences_history = []
for idx, line in enumerate(df_history_list):
  #print(f"--역사소설{idx}번--")
  preprocessed = cleansing_numbers(line)
  preprocessed = cleansing_special(preprocessed)
  #명사
  preprocessed_list = ' '.join(list(set(mecab.nouns(preprocessed))))
  #print(preprocessed_list)
  all_sentences_history.append(preprocessed_list)

In [ ]:
#로맨스
all_sentences_romance = []
for idx, line in enumerate(df_romance_list):
  #print(f"--연애소설{idx}번--")
  preprocessed = cleansing_numbers(line)
  preprocessed = cleansing_special(preprocessed)
  #명사
  preprocessed_list = ' '.join(list(set(mecab.nouns(preprocessed))))
  #print(preprocessed_list)
  all_sentences_romance.append(preprocessed_list)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from collections import defaultdict  #단어 개수를 세고, 빈도와 리스트 셋 항목 정리리

In [ ]:

print(len(all_sentences_mystery))
print(len(all_sentences_horror))
print(len(all_sentences_fantasy))
print(len(all_sentences_martial))
print(len(all_sentences_sf))
print(len(all_sentences_history))
print(len(all_sentences_romance))

2489
1230
2399
470
656
1324
2588


In [ ]:

tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(all_sentences_mystery)

word2id = defaultdict(lambda : 0)
for idx, feature in enumerate(tfidf_vectorizer.get_feature_names()):
  word2id[feature] = idx

for idx, sent in enumerate(all_sentences_mystery):
    #print(f'---미스테리 소설 {idx} 번---')
    results_mystery = [(token, tfidf_matrix[idx, word2id[token]]) for token in sent.split()]
    results_mystery.sort(key = lambda element : element[1], reverse = True)
    #print(results_mystery)
    #print("\n")

In [ ]:
#tfidf_vectorizer.vocabulary_  #어떤 단어가 어떤 번호를 할당받았는지!
tfidf_matrix.shape

(2489, 9006)

In [ ]:

#호러
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(all_sentences_horror)

word2id = defaultdict(lambda : 0)
for idx, feature in enumerate(tfidf_vectorizer.get_feature_names()):
  word2id[feature] = idx

for idx, sent in enumerate(all_sentences_horror):
  #print(f'---공포 소설 {idx} 번---')
  results_horror = [(token, tfidf_matrix[idx, word2id[token]]) for token in sent.split()]
  results_horror.sort(key = lambda element : element[1], reverse = True)
  #print(results_horror)
  #print("\n")

In [ ]:
#판타지
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(all_sentences_fantasy)

word2id = defaultdict(lambda : 0)
for idx, feature in enumerate(tfidf_vectorizer.get_feature_names()):
  word2id[feature] = idx

for idx, sent in enumerate(all_sentences_fantasy):
  #print(f'--- 판타지 소설 {idx} 번---')
  results_fantasy = [(token, tfidf_matrix[idx, word2id[token]]) for token in sent.split()]
  results_fantasy.sort(key = lambda element : element[1], reverse = True)
  #print(results_fantasy)
  #print("\n")

In [ ]:
#무협
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(all_sentences_martial)

word2id = defaultdict(lambda : 0)
for idx, feature in enumerate(tfidf_vectorizer.get_feature_names()):
  word2id[feature] = idx
  
for idx, sent in enumerate(all_sentences_martial):
  #print(f'--- 무협 소설 {idx} 번---')
  results_martial = [(token, tfidf_matrix[idx, word2id[token]]) for token in sent.split()]
  results_martial.sort(key = lambda element : element[1], reverse = True)
  #print(results_martial)
  #print("\n")

In [ ]:
#sf
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(all_sentences_sf)

word2id = defaultdict(lambda : 0)
for idx, feature in enumerate(tfidf_vectorizer.get_feature_names()):
  word2id[feature] = idx

for idx, sent in enumerate(all_sentences_sf):
  #print(f'--- sf 소설 {idx} 번---')
  results_sf = [(token, tfidf_matrix[idx, word2id[token]]) for token in sent.split()]
  results_sf.sort(key = lambda element : element[1], reverse = True)
  #print(results_sf)
  #print("\n")

In [ ]:
#역사
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(all_sentences_history)

word2id = defaultdict(lambda : 0)
for idx, feature in enumerate(tfidf_vectorizer.get_feature_names()):
  word2id[feature] = idx
  
for idx, sent in enumerate(all_sentences_history):
  #print(f'--- sf 소설 {idx} 번---')
  results_history = [(token, tfidf_matrix[idx, word2id[token]]) for token in sent.split()]
  results_history.sort(key = lambda element : element[1], reverse = True)
  #print(results_history)
  #print("\n")

In [ ]:
#연애
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(all_sentences_romance)

word2id = defaultdict(lambda : 0)
for idx, feature in enumerate(tfidf_vectorizer.get_feature_names()):
  word2id[feature] = idx
for idx, sent in enumerate(all_sentences_romance):
  #print(f'--- 연애 소설 {idx} 번---')
  results_romance = [(token, tfidf_matrix[idx, word2id[token]]) for token in sent.split()]
  results_romance.sort(key = lambda element : element[1], reverse = True)
  #print(results_romance)
  #print("\n")

In [ ]:
print(results_romance)
print(results_fantasy)
print(results_history)
print(results_horror)
print(results_martial)
print(results_sf)
print(results_mystery)

[('존경', 0.39587345523345635), ('지우', 0.381198247713804), ('태경', 0.381198247713804), ('의무', 0.33983116110493705), ('눈길', 0.33445652159932326), ('아내', 0.27055070925960334), ('남편', 0.25551242622526105), ('감정', 0.22528769231692844), ('결혼', 0.18215122968449107), ('당신', 0.17517877158723028), ('마음', 0.1579825923162859), ('여자', 0.1337867219759244), ('사랑', 0.1237592882454685), ('남자', 0.1184667274292697), ('말', 0.0), ('속', 0.0), ('수', 0.0)]
[('불로불사', 0.4464128091708481), ('고통', 0.36123067202856307), ('만큼', 0.33066898522543015), ('행복', 0.33066898522543015), ('누구', 0.26597505966738666), ('그것', 0.26020233532407483), ('상상', 0.25283910254318603), ('남자', 0.24755438959954196), ('시간', 0.23309557402676154), ('존재', 0.2248482681903135), ('자신', 0.20954138230623107), ('사람', 0.19770531939753377), ('곳', 0.0), ('태', 0.0), ('힘', 0.0)]
[('주민', 0.2497476639726848), ('제천', 0.2497476639726848), ('마고성', 0.2497476639726848), ('신시', 0.2497476639726848), ('물품', 0.2396147588609754), ('교환', 0.2396147588609754), ('신성', 0.2

# 유사도 검증 

In [18]:
# 유사도 검증에 필요한 패키지 import
import pandas as pd
from numpy import dot
from numpy.linalg import norm
import numpy as np

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity



# words(스토리)에 대해서 tf_idf 수행
tfidf = TfidfVectorizer()
tfidf_matrix_mys = tfidf.fit_transform(df_books_mystery['words'])
print(tfidf_matrix_mys.shape)

# 유사도 점수화
cosine_matrix_mys = cosine_similarity(tfidf_matrix_mys, tfidf_matrix_mys)

# 책 타이틀 
books_mys_id = {}
for i, c in enumerate(df_books_mystery['title']): books_mys_id[i] = c
        
# 책 id
id2books_mys = {}
for i, c in books_mys_id.items(): id2books_mys[c] = i


def get_recommendations(title, cosine_sim=cosine_matrix_mys):

    idx = id2books_mys[title]
    sim_scores_mys = [(i,c) for i, c in enumerate(cosine_matrix_mys[idx]) if i !=idx] 
    sim_scores_mys = sorted(sim_scores_mys, key =lambda x: x[1], reverse = True)
    sim_scores_mys[0:10]
    

    sim_scores_mys = [(books_mys_id[i], score) for i, score in sim_scores_mys[0:10]]
    return sim_scores_mys

(2489, 9006)


In [19]:
get_recommendations('단테클럽 1')

[('단테 클럽 2', 0.9744196724179254),
 ('단테 클럽 1', 0.9744196724179254),
 ('단테클럽 2', 0.256154929588162),
 ('로스트 심벌 2', 0.16397715030813564),
 ('공개수배', 0.16034001007004794),
 ('로스트 심벌 1', 0.159380637654646),
 ('살인게임', 0.15849356786754487),
 ('�A빠이, 이상', 0.1437867786664614),
 ('롱기누스의 창', 0.14301579936497277),
 ('4의 규칙 2', 0.14301080471633362)]

In [13]:
# words(스토리)에 대해서 tf_idf 수행
tfidf = TfidfVectorizer()
tfidf_matrix_mys = tfidf.fit_transform(df_books['words'])
print(tfidf_matrix_mys.shape)

# 유사도 점수화
cosine_matrix_mys = cosine_similarity(tfidf_matrix_mys, tfidf_matrix_mys)

(2489, 9006)
